## Over Time

In [2]:
import pandas as pd
import os

In [21]:
graph_folder = r"D:\66 CausalMap\SciLit_CausalMap\visualization\mir_paper"
graph_tmp = pd.read_pickle(os.path.join(graph_folder,'nodes_extracted_infos.pkl'))
graph_df = pd.read_csv(os.path.join(graph_folder,'graph_clustered.csv'))
graph_tmp['cause_topic'] = graph_df['cause']
graph_tmp['effect_topic'] = graph_df['effect']
graph_tmp['doc_id'] = graph_tmp['source'].apply(lambda x: int(x[2:]))
graph_df = graph_tmp
graph_tmp = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MIR.csv")[['Publication date', 'Month','Year','Label']]
graph_tmp['doc_id'] = graph_tmp.index
graph_df = graph_df.merge(graph_tmp, how='left', on='doc_id')
# Remove if cause==effect
graph_df = graph_df[graph_df['cause_topic']!=graph_df['effect_topic']]
print(len(graph_df))
graph_df

19885


,cause,effect,source,support,evidence,cause_action,effect_action,cause_action_rem,effect_action_rem,evidence_ner,cause_store,effect_store,cause_topic,effect_topic,doc_id,Publication date,Month,Year,Label
0,a lack of demand for their products .,"prior to ford , two more american firms - gene...",f-0,1,a lack of demand for their products . --> Prio...,a lack of demand for their products .,"prior to [ORGANIZATION] , two more american fi...",a lack of demand for their products .,"prior to [MASK] , two more american firms - [M...","[(a, O), (lack, O), (of, O), (demand, O), (for...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...",2428>>lack_products_gap_osy_addressed,695>>bert_stopped_ck_ru_orders,0,10/1/2022,Jan,2022,4
1,that there would be a rise in the demand for h...,foreign automobile manufacturers had set up th...,f-0,1,that there would be a rise in the demand for h...,that there would be a rise in the demand for h...,foreign automobile manufacturers had set up th...,that there would be a rise in the demand for h...,foreign automobile manufacturers had set up th...,"[(that, O), (there, O), (would, O), (be, O), (...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...",1667>>demand_cars_late_increasing_prices,68>>automobile_automotive_thriving_production_...,0,10/1/2022,Jan,2022,4
2,the implementation of b hara t stage vi standards,car manufacturers a bomb .,f-0,1,The implementation of B hara t Stage VI standa...,the implementation of b hara t stage vi standards,car manufacturers a bomb .,the implementation of b hara t stage vi standards,car manufacturers a bomb .,"[(The, O), (implementation, O), (of, O), (B, O...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...",2746>>bs_norms_emission_vi_meet,275>>car_makers_carmakers_bomb_european,0,10/1/2022,Jan,2022,4
3,ad d to it the rising cost of fuel,driving a car in india becomes a truly elite e...,f-0,1,Ad d to it the rising cost of fuel --> driving...,ad d to it the rising cost of fuel,driving a car in [LOCATION] becomes a truly el...,ad d to it the rising cost of fuel,driving a car in [MASK] becomes a truly elite ...,"[(Ad, O), (d, O), (to, O), (it, O), (the, O), ...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...",211>>fuel_environmental_concerns_increasing_costs,1108>>driving_claims_capability_wet_performance,0,10/1/2022,Jan,2022,4
4,the increase in the price of cars over the las...,writer and analyst ra no joy mu ker ji,f-0,1,the increase in the price of cars over the las...,the increase in the price of cars over the [DA...,writer and analyst [PERSON],the increase in the price of cars over the [MA...,writer and analyst [MASK],"[(the, O), (increase, O), (in, O), (the, O), (...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...",233>>prices_hike_car_vehicles_kes,1230>>writer_researcher_analyst_researchers_pr...,0,10/1/2022,Jan,2022,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20170,", sees china ' s shortage continuing through 2...",and de ems it dangerous to hold inventory afte...,f-1002,1,", sees China ' s shortage continuing through 2...",", sees [LOCATION] ' s shortage continuing thro...",and de ems it dangerous to hold inventory afte...,", sees [MASK] ' s shortage continuing through ...",and de ems it dangerous to hold inventory afte...,"[(,, O), (sees, O), (China, LOCATION), (', O),...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...","{'ORGANIZATION': [], 'LOCATION': [], 'DATE': [...",2145>>prolonged_sees_continuing_shortage_short...,2299>>ems_dangerous_inventory_hold_raiding,1002,18/7/2022,Jul,2022,2
20171,to stem from the merger between the italian am...,", the group set",f-1003,1,to stem from the merger between the Italian Am...,to stem from the merger between the i

In [28]:
# randomly sample by time
y2022_indexes = []
y2020_2021_indexes = []
others_indexes = []

for i,year in enumerate(list(graph_df['Year'])):
    if int(year)==2022:
        y2022_indexes.append(i)
    elif int(year)>=2020:
        y2020_2021_indexes.append(i)
    else:
        others_indexes.append(i)

n_samples = [len(y2022_indexes),len(y2020_2021_indexes),len(others_indexes)]
min_n = min(n_samples)
print(f'min {min_n}, from {n_samples}')

min 2668, from [10844, 6373, 2668]


In [29]:
keep_indexes = others_indexes + list(np.random.choice(y2022_indexes, min_n)) + list(np.random.choice(y2020_2021_indexes, min_n))
print(len(keep_indexes))

8004


In [34]:
subgraph = graph_df.iloc[keep_indexes]
sG = get_graph_from_df(subgraph)
nx.write_gml(sG, os.path.join(graph_folder, "graph_random_sample.gml"))

num_edges 6799
num_nodes 2833


In [45]:
import networkx as nx
import os
import re
import pandas as pd
import numpy as np
from collections import Counter
from copy import copy


def get_graph_from_df(graph_df):
    node_weights = pd.concat([
        graph_df[['cause_topic','support']].rename(columns={'cause_topic':'node'}),
        graph_df[['effect_topic','support']].rename(columns={'effect_topic':'node'})
    ],axis=0)
    node_weights = node_weights.groupby('node')['support'].sum().reset_index()
    node_weights = node_weights.sort_values(by='support', ascending=False)
    node_weights = dict(zip(node_weights.node, node_weights.support))

    G = nx.DiGraph()
    for node, weight in node_weights.items():
        G.add_node(
            node.split('>>')[-1], 
            weight=weight,
            pandemic='pandemic' if 'pandemic' in node.lower() else 'not',
            evs='EVs' if bool(re.search("(electr.*vehic.*)|( ev )|( evs )||(_ev_)|(_evs_)|(vehic.*electr.*)", node.lower())) else 'not',
            chipshortage='chipshortage' if bool(re.search("(chip.*shortag.*)|(shortag.*chip.*)", node.lower())) else 'not',
            robotics='robotics' if bool(re.search("(robot.*)", node.lower())) else 'not'
        )

    for i,row in graph_df.iterrows():
        G.add_edge(
            row.cause_topic.split('>>')[-1], 
            row.effect_topic.split('>>')[-1], 
            weight=row.support,
            evidence=str(row.evidence),
            source=str(row.source),
            year='y22' if int(row.Year)==2022 else 'y20,21' if int(row.Year)>=2020 else 'others'
        )
    
    print('num_edges', len(G.edges))
    print('num_nodes', len(G.nodes))
    
    return G

In [24]:
from collections import Counter
Counter(graph_df['Year'])

Counter({2022: 10844,
         2021: 4689,
         2018: 806,
         2019: 1128,
         2020: 1679,
         2017: 733,
         2011: 1,
         2025: 5})

## With Doc Infos

In [49]:
import networkx as nx
import os
import re
import pandas as pd
import numpy as np
from collections import Counter
from copy import copy


def get_graph_from_df(graph_df):
    node_weights = pd.concat([
        graph_df[['cause_topic','support']].rename(columns={'cause_topic':'node'}),
        graph_df[['effect_topic','support']].rename(columns={'effect_topic':'node'})
    ],axis=0)
    node_weights = node_weights.groupby('node')['support'].sum().reset_index()
    node_weights = node_weights.sort_values(by='support', ascending=False)
    node_weights = dict(zip(node_weights.node, node_weights.support))

    G = nx.DiGraph()
    for node, weight in node_weights.items():
        G.add_node(
            node.split('>>')[-1], 
            weight=weight,
            pandemic='pandemic' if 'pandemic' in node.lower() else 'not',
            evs='EVs' if bool(re.search("(electr.*vehic.*)|( ev )|( evs )||(_ev_)|(_evs_)|(vehic.*electr.*)", node.lower())) else 'not',
            chipshortage='chipshortage' if bool(re.search("(chip.*shortag.*)|(shortag.*chip.*)", node.lower())) else 'not',
            robotics='robotics' if bool(re.search("(robot.*)", node.lower())) else 'not'
        )

    for i,row in graph_df.iterrows():
        G.add_edge(
            row.cause_topic.split('>>')[-1], 
            row.effect_topic.split('>>')[-1], 
            weight=row.support,
            evidence=str(row.evidence),
            article=str(row.source)
        )
    
    print('num_edges', len(G.edges))
    print('num_nodes', len(G.nodes))
    
    return G

In [58]:
graph_folder = r"D:\66 CausalMap\SciLit_CausalMap\visualization\mir_paper"
graph_tmp = pd.read_pickle(os.path.join(graph_folder,'nodes_extracted_infos.pkl'))
graph_df = pd.read_csv(os.path.join(graph_folder,'graph_clustered.csv'))
graph_tmp['cause_topic'] = graph_df['cause']
graph_tmp['effect_topic'] = graph_df['effect']
graph_tmp['doc_id'] = graph_tmp['source'].apply(lambda x: int(x[2:]))
graph_df = graph_tmp
graph_tmp = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MIR.csv")[['Publication date', 'Month','Year','Label']]
graph_tmp['doc_id'] = graph_tmp.index
graph_df = graph_df.merge(graph_tmp, how='left', on='doc_id')
graph_df['source'] = graph_df['source'].apply(lambda x: "f-{:05d}".format(int(x[2:])))

# Remove if cause==effect
graph_df = graph_df[graph_df['cause_topic']!=graph_df['effect_topic']]
print(len(graph_df))

# # Group cause-effect pairs
# graph_df = graph_df.groupby(['cause_topic','effect_topic'])[['source','support','evidence','doc_id']].agg({
#     'source': ', '.join,
#     'evidence': '/n'.join,
#     'support': sum
# }).reset_index()
# graph_df = graph_df.sort_values(by='support')
print('unique sents/edges:',len(graph_df))
G = get_graph_from_df(graph_df)

nx.write_gml(G, os.path.join(graph_folder, "graph_w_docinfos.gml"))

19885
unique sents/edges: 19885
num_edges 17801
num_nodes 3000


In [57]:
graph_df['source']

0                                                  f-03950
11584                                              f-05060
11585                                              f-05502
11586                                              f-05866
11587                                              f-04075
                               ...                        
1271     f-00199, f-00199, f-00199, f-02145, f-02145, f...
3357     f-05889, f-05892, f-05893, f-05894, f-06045, f...
16154    f-02863, f-02863, f-02863, f-02863, f-02903, f...
7955     f-05892, f-05894, f-05894, f-06045, f-06046, f...
5954     f-05889, f-05892, f-05893, f-05894, f-06046, f...
Name: source, Length: 17801, dtype: object